In [1]:
source("evaluate_SC2.R")
source("utils_SC2.R")
source("predictors_SC2.R")

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-18

Loading required package: prodlim

 randomForestSRC 2.9.3 
 
 Type rfsrc.news() to see new features, changes, and bug fixes. 
 

Loading required package: RColorBrewer


In [2]:
#features_model<- list(c("RNA"),c("DNA"),c("clinical_categorical"),c("clinical_numerical"),c("clinical_numerical_reduced"),c("AUC"),
#                     c("RNA","DNA"),c("RNA","clinical_categorical","clinical_numerical_reduced"),c("RNA","AUC"),c("DNA","clinical_categorical","clinical_numerical_reduced"),
#                     c("DNA","AUC"),c("clinical_categorical","clinical_numerical_reduced","AUC"),
#                     c("RNA","DNA","clinical_categorical","clinical_numerical_reduced"),c("RNA","DNA","AUC"),c("DNA","clinical_categorical","clinical_numerical_reduced","AUC"),
#                     c("RNA","DNA","clinical_categorical","clinical_numerical_reduced","AUC"))

# TESTED :
# Conclusion: RNA_hr > clin-num ~= mol > RNA > RNA_var > AUC
#features_model <- list(c("clinical_numerical"),c("mol"),c("RNA"),c("RNA_var"),c("RNA_hr"),c("AUC"))

features_model <- list(c("clinical_numericalwithmol"))

predictors <- c(predictorGLM,predictorRF)
str_predictors <- c("predictorGLM","predictorRF")

# Specify the parameters you want :

# Parameters for ElasticNet:
l_alpha <-seq(0,1,0.1)

# Parameters for RF:
l_ntree <- seq(10,700,30)
l_nodesize <- seq(5,25,4) 

# Parameters for Cox Boost
l_maxstepno=seq(100,900,100) 
l_K=10 
l_type="verweij" 
l_penalty=c(0.01,0.1,1,5,10,20,40,80,100,150,200)

# Parameters for Random Effects
l_max.iter = seq(100,900,100)
l_tol=c(0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.2)

launch_prediction_SC2(features_model,  
                 predictors, str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,
                     l_nodesize=l_nodesize,l_maxstepno=l_maxstepno,l_K=l_K,l_type=l_type,
                     l_penalty=l_penalty,l_max.iter=l_max.iter,l_tol=l_tol )

[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/clinical_numericalwithmol/stdoutput/predictorGLM_0_0.txt' 'Rscript evaluate_SC2_intermediate_parallel.R --predictor predictorGLM --feature_model clinical_numericalwithmol --use_MSK 0 --alpha 0 --nrepeats 10 --nfolds 5 --mc.cores 4 --outpath ./../prediction/results_SC2/clinical_numericalwithmol --filename predictorGLM_0_0'"
[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/clinical_numericalwithmol/stdoutput/predictorGLM_0.1_0.txt' 'Rscript evaluate_SC2_intermediate_parallel.R --predictor predictorGLM --feature_model clinical_numericalwithmol --use_MSK 0 --alpha 0.1 --nrepeats 10 --nfolds 5 --mc.cores 4 --outpath ./../prediction/results_SC2/clinical_numericalwithmol --filename predictorGLM_0.1_0'"
[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/clinical_numericalwithmol/stdoutput/predictorGLM_0.2_0.txt' 'Rscript evaluate_SC2_intermedia

In [3]:
# THIS ONE is now training on the combined data but testing only on most similar part of data challenge
features_model<- list(c("combined_features"))

l_use_MSK <- seq(0,100,10) # percentage of most similar data to use

launch_prediction_SC2(features_model, 
                 predictors, str_predictors,l_alpha=l_alpha,l_ntree=l_ntree, l_maxstepno=l_maxstepno,l_K=l_K,l_type=l_type, 
                     l_penalty=l_penalty,l_max.iter=l_max.iter,l_tol=l_tol,l_use_MSK=l_use_MSK,
                     l_nodesize=l_nodesize)

[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/combined_features/stdoutput/predictorGLM_0_0.txt' 'Rscript evaluate_SC2_intermediate_parallel.R --predictor predictorGLM --feature_model combined_features --use_MSK 0 --alpha 0 --nrepeats 10 --nfolds 5 --mc.cores 4 --outpath ./../prediction/results_SC2/combined_features --filename predictorGLM_0_0'"
[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/combined_features/stdoutput/predictorGLM_0_10.txt' 'Rscript evaluate_SC2_intermediate_parallel.R --predictor predictorGLM --feature_model combined_features --use_MSK 10 --alpha 0 --nrepeats 10 --nfolds 5 --mc.cores 4 --outpath ./../prediction/results_SC2/combined_features --filename predictorGLM_0_10'"
[1] "bsub -R 'rusage[mem=4]' -W 05:59 -n 4 -J 'model' -o '../prediction/results_SC2/combined_features/stdoutput/predictorGLM_0_20.txt' 'Rscript evaluate_SC2_intermediate_parallel.R --predictor predictorGLM --feature_model combi

In [8]:
df<-get_models(path = "../prediction/results_SC2",dataset="combined_features")
response <- read.table("../data/responses/prepared_data_response.csv")
#write.table(df,"../prediction/results_SC2/SC2.csv",sep="\t")

In [9]:
best_model <- df[order(df$mean,decreasing=TRUE),][1,]
best_model
save_trained_best_model_SC2(best_model,response,path.features="./../data/features_SC2/")

,mean,sd,min,max
,<dbl>,<dbl>,<dbl>,<dbl>
/combined_features/predictorRF_100_21_100.csv,0.6871136,0.04750081,0.5649819,0.8041616


[1] 293  32
[1] "a"
[1] "a"


In [51]:
#celllines <- read.table("./../data/features_SC2/prepared_data_rna_vargenes.csv",sep="")

celllines <-as.data.frame(combine_features(c("combined_features")))

params <- preProcess(celllines, method = "range")
celllinesb <- predict(params, celllines)
celllinesb

response <- read.table("../data/responses/prepared_data_response.csv")
result <- evaluateCV_SC2(predictorGLM, celllinesb, response, alpha=0, nrepeats=10, nfolds=5, mc.cores=4, use_MSK=0)
df_tosave <- data.frame(mean=mean(result),sd=sd(result),min=min(result),max=max(result))
df_tosave

[1] 193  32


,ASXL1,BCOR,DNMT3A,FLT3,IDH1,IDH2,JAK2,KIT,KRAS,NPM1,⋯,inv_16,inv_3,CEBPA_bi,t_8_21,t_9_11,t_15_17,gender,bm_blasts,wbc,age
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
13-00118,0,0,1,1,0,0,0,0,0,1,⋯,0,0,0,0,0,0,1,0.84693878,0.13334401,0.61176471
13-00149,0,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,1,0.73469388,0.03203460,0.89411765
13-00157,0,0,0,0,1,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0.32653061,0.21383094,0.68235294
13-00160,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,1,0.46154740,0.15779296,0.80000000
13-00165,0,0,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0.46154740,0.15779296,0.36470588
13-00166,0,0,1,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,1,0.78571429,0.07608217,0.74117647
13-00186,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,0.29591837,0.04524887,0.84705882
13-00195,0,0,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,1,0.24489796,0.11011893,0.75294118
13-00204,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0.58163265,0.03724022,0.74117647


[1] "start CV"


mean,sd,min,max
<dbl>,<dbl>,<dbl>,<dbl>
0.6358692,0.06353198,0.5471014,0.7561837


In [3]:
df_rna_vargenes2 <- read.table("./../data/features_SC2/prepared_data_rna_vargenes.csv",sep="")
df_rna_vargenes1 <- read.table("./../data/features/prepared_data_rna_vargenes.csv",sep="")
all.equal(df_rna_vargenes1,df_rna_vargenes2)

[1] TRUE

In [17]:
response <- read.table("../data/responses/prepared_data_response.csv")
response
combined_features <- read.table("./../data/features_SC2/prepared_data_challenge_combined.csv",sep="")
combined_features

,OS,OS_status
,<int>,<int>
13-00098,NA,NA
13-00118,1378,1
13-00149,267,1
13-00157,1379,0
13-00160,1165,1
13-00165,439,1
13-00166,1020,1
13-00186,126,1
13-00195,972,0


,ASXL1,BCOR,DNMT3A,FLT3,IDH1,IDH2,JAK2,KIT,KRAS,NPM1,⋯,inv_16,inv_3,CEBPA_bi,t_8_21,t_9_11,t_15_17,gender,bm_blasts,wbc,age
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>
13-00118,0,0,1,1,0,0,0,0,0,1,⋯,0,0,0,0,0,0,1,83.00000,33.80000,54
13-00149,0,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,1,72.00000,8.50000,78
13-00157,0,0,0,0,1,0,0,0,0,1,⋯,0,0,0,0,0,0,0,32.00000,53.90000,60
13-00160,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,1,45.23164,39.90564,70
13-00165,0,0,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,45.23164,39.90564,33
13-00166,0,0,1,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,1,77.00000,19.50000,65
13-00186,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,29.00000,11.80000,74
13-00195,0,0,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,1,24.00000,28.00000,66
13-00204,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,57.00000,9.80000,65


In [31]:
y <- read.table("./../data/features_SC2/prepared_data_clin_cat_mut_clean.csv",sep="\t") #mol
colnames(y)[c(3,4,9:27)]
#colnames(y)
x <- read.table("./../data/features_SC2/save_data_dna_total.csv",sep="\t")
x
colnames(x[,which(colMeans(x) > 0.03)])

[1] "CEBPAbi" "NPM1"    "ITD"     "AGTPBP1" "ASXL1"   "BCOR"    "DNMT3A" 
 [8] "FLT3"    "IDH1"    "IDH2"    "JAK2"    "KRAS"    "NRAS"    "RUNX1"  
[15] "SF3B1"   "SRSF2"   "TET2"    "TP53"    "TRPM3"   "U2AF1"   "WNK1"

,A1CF,AARS2,ABCA9,ABI3BP,ACAN,ACSM2A,ACTL6B,ADAMTS12,ADAMTS7,ADAMTSL1,⋯,WT1,XIRP1,YIPF5,ZBTB47,ZFHX4,ZNF195,ZNF285,ZNF518B,ZNF711,ZZEF1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
13-00098,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00118,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00149,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00157,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00160,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00165,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00166,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00186,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
13-00195,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] "AGTPBP1" "ASXL1"   "BCOR"    "DDX60L"  "DNMT3A"  "FLT3"    "IDH1"   
 [8] "IDH2"    "IMPG1"   "JAK2"    "KRAS"    "NPM1"    "NRAS"    "RUNX1"  
[15] "SF3B1"   "SRSF2"   "TET2"    "TP53"    "TPRX1"   "TRIO"    "TRPM3"  
[22] "U2AF1"   "WNK1"    "ZNF711"

In [26]:
colnames(x[,which(colMeans(x) > 0.03)])

[1] "AGTPBP1" "ASXL1"   "BCOR"    "DDX60L"  "DNMT3A"  "FLT3"    "IDH1"   
 [8] "IDH2"    "IMPG1"   "JAK2"    "KRAS"    "NPM1"    "NRAS"    "RUNX1"  
[15] "SF3B1"   "SRSF2"   "TET2"    "TP53"    "TPRX1"   "TRIO"    "TRPM3"  
[22] "U2AF1"   "WNK1"    "ZNF711"

In [ ]:
x